In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import random
import nltk
from operator import itemgetter
import re
import numpy as np
import math

In [2]:
def textProc(file_name):
    f= open(file_name,'r')
    #tp1
    corpus = f.read().replace('\n',' ') #replace the new line with single white space.
    corpus = corpus.lower() #tp2

    token = sent_tokenize(corpus)

    #replace all the non alphanumeric characters with whitespace.
    token = [re.sub(r'[^A-Za-z0-9]'," ", i) for i in token]#tp3

    random.shuffle(token)

    #addtG(token) #add tags
    return token

    #clean tokens of sentence returned


In [3]:
# function for count of n-gram
def LstOfNgram(count,corpus):
    #cleaned corpus
    n_gram=[]
    for i in corpus:
        p = word_tokenize(i)
        p = ["<s>"]+p
        p.append("</s>")    
        #token must have start and end line tags here.
        #print (p)
        for g in range(count-1,len(p)):
            dummy=""
            for j in range(count):
                dummy = p[g-j]+" "+dummy   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
               #more things to be added here
            dummy = re.sub(r"^\s|\s$","",dummy)
            n_gram.append(dummy)
    return (n_gram)    

    

In [4]:
def CountNgram(frequency):
    count ={}
    
    for (key,fvalue) in reversed(sorted(frequency.items(), key = itemgetter(1))):
        count[key] = fvalue
 
    return count

In [5]:
#unified method for MLE of n-gram
def MLE(a,b,N):  #n is N of n-gram
    try:
        if N ==1:#unigram
            return (count_unigram[a]/len(unigramLst))
        elif N==2:#bigram
            return (count_bigram[b+" "+a]/count_unigram[b])
        elif N == 3:#triram
            return (count_trigram[b+" "+a]/count_bigram[b])
        elif N == 4:
            return (count_quadgram[b+" "+a]/count_trigram[b])
    except:
            return 0 #0 probability if not present

In [6]:
def get_NLst(count_dict):#pass dicationary of word vs count.
    #create counting list for 
    l = count_dict 
    l=list(l.values())
    l.sort()
    #we want a dictionary with {1:1,2:1,3:2,4:3,5:1}
    d={}
    for i in range(len(l)):
        count =1
        for j in range(i,0,-1):
            if l[i] == l[j-1]:
                count = count + 1
            else:
                break 
    
        d[l[i]] = count
    return (d)
#takes O(n^2) which takes lot of time to execute.
    

In [7]:
class add1_smoothing_bigram:#old_count is the dictionary of count versus bigram/unigram
    
    def __init__(self,old_countUnigram, old_countBigram):
        self.old_countUnigram = old_countUnigram.copy()
        self.old_countBigram = old_countBigram.copy()
        self.new_countBigram = old_countBigram.copy()
#        print (self.new_countBigram)
        self.countBigram()
        
        
    def countBigram(self):
           
            for i in self.new_countBigram.keys():
                a_,b_= i.split(" ")[1],i.split(" ")[0]
                self.new_countBigram[i]=self.old_countUnigram[b_]*self.Prob(a_,b_,1)
                
        
    def Prob(self,a,b,countBigram): #P(a|b)
            bigram = b+" "+a
            unigram = b
            try:
                numerator = self.old_countBigram[bigram] +1
            except :
                numerator = 1
            try:   
                denominator = self.old_countUnigram[unigram] + len(self.old_countUnigram.keys()) #types
            except :
                denominator = len(self.old_countUnigram.keys())
            return (numerator/denominator)
#assure once that it is len(count_bigam)    #see if it needs to be replaced by token?

In [8]:
class goodTuring:
    def __init__(self,Bcount,Ucount): #dictionary of unigram and bigram count
        self.biGramCount = Bcount.copy()
        self.UniGramCount = Ucount.copy()
        
        self.bidictN = get_NLst(self.biGramCount)
        self.UnidictN = get_NLst(self.UniGramCount)
        
        self.keysbi,self.valuesbi = list(self.bidictN.keys()),list(self.bidictN.values())    #[N1,N2,N3,N4,N5,N6,...]
        self.keysuni,self.valuesuni = list(self.UnidictN.keys()),list(self.UnidictN.values()) #count of N1,N2,N3,...
        
        self.NewCountbi()
        self.NewCountuni()
    def NewCountbi(self):
               
        for i in self.biGramCount.keys():
            try:
                self.biGramCount[i] = (self.biGramCount[i]+1)*(self.bidictN[self.biGramCount[i]+1]/self.bidictN[self.biGramCount[i]])
            except:
                intr= np.interp(float(self.biGramCount[i]),[float(j) for j in self.keysbi],[float(k) for k in self.valuesbi])
                #intr= np.interp(l[i]+1,keys,values)

                self.biGramCount[i] = (self.biGramCount[i]+1)*(intr/(self.bidictN[self.biGramCount[i]]))
            #new for the word with zero count 

        return self.biGramCount 
    def NewCountuni(self):
               
        for i in self.UniGramCount.keys():
            try:
                self.UniGramCount[i] = (self.UniGramCount[i]+1)*(self.UnidictN[self.UniGramCount[i]+1]/self.UnidictN[self.UniGramCount[i]])
            except:
                intr= np.interp(float(self.UniGramCount[i]),[float(j) for j in self.keysuni],[float(k) for k in self.valuesuni])
                #intr= np.interp(l[i]+1,keys,values)

                self.UniGramCount[i] = (self.UniGramCount[i]+1)*(intr/(self.UnidictN[self.UniGramCount[i]]))
            #new for the word with zero count 

        return self.UniGramCount 
    
    def Prob(self,a,b, tokenBigram):   #total number of bygram needs to be passed here
        #token : total number of token of n-gram 
        bigram = b+" "+a
        if bigram not in self.biGramCount.keys():
            
            return (self.bidictN[1]/len(tokenBigram))
        
        return self.biGramCount[bigram]/self.UniGramCount[b]
#look over interpolation function
#for Nc >Nc+1 it is too jumpy
#the above probability may be wrong??
        

In [48]:

def perPlexity(data,bigramLst,model):
    log_perplxty = 0
    for i in data:#data is the list of all the sentences 
        #i is a sentence
        log_prob = 0
        a = LstOfNgram(2,[i])
        for i in a:
            new = model.Prob(i.split(" ")[1],i.split(" ")[0],bigramLst)
            #print(new)
            log_prob = log_prob + math.log(new)
            
        log_perplxty = log_perplxty+log_prob
        log_perplxty = (-1/len(data))*(log_perplxty)
       
    return (math.exp(log_perplxty) )



In [50]:
token = textProc('SherlockHomes.txt')
random.shuffle(token)

train = token[:int(len(token)*0.8)]#80%
test= token[int(len(token)*0.8):]#20%

In [51]:
#n-gram count

#list contining n-gram
unigramLst = LstOfNgram(1,train)  ##the first number here representes the N of N-gram.
bigramLst  = LstOfNgram(2,train)
trigramLst = LstOfNgram(3,train)
quadgramLst= LstOfNgram(4,train)
#print (unigramLst)
#  query for count of unigram only once
feq  =  nltk.FreqDist(unigramLst)
freq =  nltk.FreqDist(bigramLst)
fre  =  nltk.FreqDist(trigramLst)
fr   =  nltk.FreqDist(quadgramLst)

count_unigram = CountNgram(feq)
count_bigram = CountNgram(freq)
count_trigram = CountNgram(fre)
count_quadgram = CountNgram(fr)
#print (count_)


In [52]:
print (len(bigramLst))

90608


# MLE for unigram, bigram, trigrams and quadgrams. 

In [53]:
l1 = MLE('room','',1)
l2 = MLE('room','the',2)
l3 = MLE('room','into the',3)
l4 = MLE('room', 'rushed into the',4)
print (l1," ",l2," ",l3," ",l4)


0.0013950921906070734   0.010118785745710514   0.12871287128712872   0.5714285714285714


# Smoothing : ADD-1 

In [54]:
#QUESTION 5
ADD1 = add1_smoothing_bigram(count_unigram,count_bigram)
newProb = ADD1.Prob("affair","strange",1)
#print (ADD1.new_countBigram)
#print(newProb)
#print(count_bigram)
#work for sentence probability

# Smoothing : Good-turing

In [55]:
#n-gram evalutation, how many n-grams are there
#question 6 

p = goodTuring(count_bigram,count_unigram)

#goodTuning smooting trained


In [56]:
GT_bigramCount = p.biGramCount
GT_unigramCount = p.UniGramCount

##  Constant value discounting in good-tunning, for count b/w 1 and 10

In [57]:

#print (smooth_count)
print (count_bigram["roylott had"] -GT_bigramCount["roylott had"])#n=1
print (count_bigram['merry over'] - GT_bigramCount['merry over'])#n=2
print (count_bigram['be taken'] - GT_bigramCount['be taken'])
print (count_bigram["glass of"] - GT_bigramCount["glass of"])
print (count_bigram['be taken'] - GT_bigramCount['be taken'])

print (count_bigram['is this'] - GT_bigramCount['is this'])
print (count_bigram['the bird'] - GT_bigramCount['the bird'])#n =7
print (count_bigram['holmes said'] - GT_bigramCount['holmes said'])
print (count_bigram['until i'] - GT_bigramCount['until i'])
print (count_bigram['other side'] - GT_bigramCount['other side'])
#the vaue of d is not constant in this corpus, after smoothing.

0.6857950395376163
0.8314129775621588
1.0464362850971924
1.0778412039439544
1.0464362850971924
0.9744058500914079
0.9744058500914079
2.5064935064935066
0.5982905982905979
0.5982905982905979


## the constant value of d is close to 0.8.

# Perplexity - Model Measure 

In [58]:

BigramLst  = LstOfNgram(2,test)
UnigramLst = LstOfNgram(1,test)
Freq =  nltk.FreqDist(BigramLst)
count_Bigram = CountNgram(Freq)
l = perPlexity(BigramLst,bigramLst,p)
print (l)
m = perPlexity(BigramLst,bigramLst,ADD1)
print (m)


1.0002383872811003
1.0020284690912606


In [ ]:
## ADD-1 is a poor model compared to good turing accoring to the perplexity values.

In [ ]:
#rough work below[a]/count_unigram

In [21]:
l="<s>this is a sentence here is.</s>"
token = word_tokenize(l)
#print (token)
regx = re.compile("[a-z0-9]+ [a-z0-9]+")
r =re.findall(regx,l)
p= ["<s>this", "this is","is a ","a sentence","sentence here","here is","is. </s>"]
print (r)

['this is', 'a sentence', 'here is']


In [22]:



import re
l = "this is a sentence here is"
token1 = word_tokenize(l)
token1 = ["<s>"]+token1
token1.append("</s>")
n_gram = []
n=2 #bigram 
#so keeping a sentence token is a good idea
#for i in range(n-1,len(token)):#start with the n-1 of the n gram, so that it does not give any error
 #   n_gram.append(token[i-1]+" "+token[i])
#print (n_gram)

#for n gram              <s>this is a sentence here is</s>
print (token1)
for i in range(n-1,len(token1)):
    p=" "
    for j in range(n):
       p = token1[i-j]+" "+p   #i,i-1,i-2,i-3,i-4,i-5,...,i-(n-1)
       #more things to be added here
    n_gram.append(p)
#print (n_gram)

['<s>', 'this', 'is', 'a', 'sentence', 'here', 'is', '</s>']


In [23]:
#testing goods tuning
l = ["this is a sentence","and I am shoked that it is","a sentence for you"]
uni = LstOfNgram(1,l)
bi = LstOfNgram(2,l)
f = nltk.FreqDist(bi)
feq  =  nltk.FreqDist(uni)
count_unigr = CountNgram(feq)
count_bi = CountNgram(f)
p = goodTuring(count_bi,count_unigr)

print (bi)
print(p.Prob("is","this",bi))


['<s> this', 'this is', 'is a', 'a sentence', 'sentence </s>', '<s> and', 'and I', 'I am', 'am shoked', 'shoked that', 'that it', 'it is', 'is </s>', '<s> a', 'a sentence', 'sentence for', 'for you', 'you </s>']
0.1875
